In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
def get_level_agg_metric(metrics,series_names):
    level_agg_crps = {}
    level_num_list = [len(n.split('_')) for n in series_names]
    for level_num in list(set(level_num_list)):
        idx_list = [i for i,x in enumerate(level_num_list) if x==level_num]
        try:
          level_agg_crps[level_num] = np.array(metrics['CRPS'])[idx_list].mean()
        except:
          level_agg_crps[level_num] = np.array(metrics['crps'])[idx_list].mean()
    try:
      avg_crps = metrics['CRPS'].mean()
    except:
      avg_crps = metrics['crps'].mean()
    return level_agg_crps, avg_crps

In [3]:
hier_dict_name = np.load('data/tourism/hier_dict_name.npy',allow_pickle=True).item()
hier_dict = np.load('data/tourism/hier_dict.npy',allow_pickle=True).item()
series_names = list(pd.read_csv('data/tourism/series_names.csv',index_col=0)['0'])
df_l = []
avg_crps_l = []

search_params = {
        'lstm_dropout': np.arange(0, 0.501, 0.2).tolist(),
        'lam':np.arange(0, 0.01, 0.002).tolist()
    }
keys = sorted(search_params.keys())
search_range = list(itertools.product(*[[*range(len(search_params[i]))] for i in keys]))

for search_range_ in [(i, ) for i in search_range]:
  sr = search_range_ [0]
  params = {k: search_params[k][sr[idx]] for idx, k in enumerate(sorted(search_params.keys()))}
  model_param_list = '-'.join('_'.join((k, f'{v}')) for k, v in params.items())

  try:
    my_metrics = np.load(f'experiments/param_search/{model_param_list}/eval_metrics.npy',allow_pickle=True).item()
    level_agg_crps,avg_crps = get_level_agg_metric(my_metrics,series_names)
    df = pd.DataFrame(level_agg_crps,index=[f'{model_param_list}']).T
    avg_crps_l.append(avg_crps)
    df_l.append(df)
  except:
    continue

In [4]:
min(avg_crps_l)

21.728591857947034

In [5]:
avg_crps_l

[21.728591857947034,
 22.462724026940492,
 24.570310280284307,
 24.282212271037654,
 29.969864391461414,
 27.547245313763725,
 28.90866194783905,
 35.277833035759514,
 28.948995355692023]

In [6]:
res_df = pd.concat(df_l,axis=1)

In [7]:
res_df

,lam_0.0-lstm_dropout_0.0,lam_0.0-lstm_dropout_0.4,lam_0.002-lstm_dropout_0.2,lam_0.002-lstm_dropout_0.4,lam_0.004-lstm_dropout_0.0,lam_0.004-lstm_dropout_0.4,lam_0.006-lstm_dropout_0.2,lam_0.008-lstm_dropout_0.0,lam_0.008-lstm_dropout_0.4
1,924.028723,1026.469002,1178.718103,1148.182953,2117.863463,1635.103809,1873.506759,3010.898985,2072.580482
2,187.466772,198.388743,213.152242,206.430301,308.715431,252.601969,278.871756,408.288693,290.202663
3,32.357973,32.892545,37.577420,36.085930,43.114139,39.861392,42.535400,48.008844,41.290355
4,12.374924,12.597189,13.285347,13.542525,13.569794,14.134489,13.832043,13.956453,13.290905


In [12]:
res_df.to_csv('params_search.csv')

In [19]:
np.arange(0, 0.01, 0.002).tolist()

[0.0, 0.002, 0.004, 0.006, 0.008]

In [5]:
level_agg_crps

{1: 26221.669921875, 2: 1953.0647695270761, 3: 345.158898775539}